In [6]:
import pandas as pd
import numpy as np
import datetime
import cc_vars as cc

file =  'Customer Service Daily AHT Volume.xlsx'

def clean_daily_vol_aht(f,g,tab):
    df = pd.read_excel(f, sheet_name = tab).fillna(0)
    df.index = df['Split']
    df.insert(0,'Forecast_Grp',df['Split'].map(g))
    df = df.loc[df['Forecast_Grp'].isna() == False]
    df = df.sort_values(by=['Forecast_Grp'])
    save =  pd.DataFrame(df['Forecast_Grp'],index=df.index)
    df = df.drop(['Forecast_Grp','Split','Group'],axis=1)
    df.columns = pd.to_datetime(df.columns)
    df = df.loc[:, ~df.columns.weekday.isin([5,6])]
    df.insert(0,'Forecast_Grp',save)
    df = df.transpose()
    return df

def subtract_week(t,hols,c = 3):
    modified_date = t - datetime.timedelta(weeks = c)
    while True:
        if modified_date in hols:
            modified_date -= datetime.timedelta(weeks = 1)
        else:
            break
    return modified_date

def add_day(t,c=1):
    modified_date = t + datetime.timedelta(days = c)
    return modified_date

def update_hol_lis(hols):
    extra_days = []
    for h in hols:
        extra_days.append(add_day(h))
        extra_days.append(add_day(h,2))
        extra_days.append(add_day(h,3))
    new = hols + extra_days
    return new

def empty_date_df(df,start_date,
                  days_to_forecast = 30):
    end_date = datetime.datetime.now().strftime("%Y-%m-%d")
    end_date = pd.to_datetime(end_date)
    delta = int((end_date - start_date).days) + days_to_forecast
    arr = np.array([start_date + datetime.timedelta(days=i) for i in range(delta)])
    cols = df.columns
    return pd.DataFrame(columns = cols, index = arr).fillna(0)

def forecast_df(empty_df,g,full_df):
    forc = empty_df
    full_df2 = full_df.loc[full_df.index != 'Forecast_Grp']
    forc.loc[full_df2.index] = full_df2   
    grp_row = full_df.loc[full_df.index == 'Forecast_Grp']
    return forc, grp_row

def forecast(vol_df, aht_df, g, hols, start_date, 
                 days_to_forecast = 30):
    h = update_hol_lis(hols)
    last = vol_df.index[-1]
    vol_forc, vg = forecast_df(empty_date_df(vol_df,start_date,days_to_forecast),g,vol_df)
    aht_forc, ag = forecast_df(empty_date_df(aht_df,start_date,days_to_forecast),g,aht_df)

    for days in range(1,days_to_forecast+1):
        d = add_day(last,days)
        if d in hols:
            pass
        elif d.to_pydatetime().weekday() > 4:
            pass
        elif add_day(d,-1) in h or add_day(d,-2) in h:
            vol_forc.loc[d] = ((vol_forc.loc[subtract_week(d,h,3)]+
                              2*vol_forc.loc[subtract_week(d,h,2)]+
                              3*vol_forc.loc[subtract_week(d,h,1)])//6)*1.2
            aht_forc.loc[d] = ((aht_forc.loc[subtract_week(d,h,3)]+
                              2*aht_forc.loc[subtract_week(d,h,2)]+
                              3*aht_forc.loc[subtract_week(d,h,1)])//6)*1.2
        else:
            vol_forc.loc[d] = (vol_forc.loc[subtract_week(d,h,3)]+
                              2*vol_forc.loc[subtract_week(d,h,2)]+
                              3*vol_forc.loc[subtract_week(d,h,1)])//6

            aht_forc.loc[d] = (aht_forc.loc[subtract_week(d,h,3)]+
                                2*aht_forc.loc[subtract_week(d,h,2)]+
                                3*aht_forc.loc[subtract_week(d,h,1)])//6

    return pd.concat([vg,vol_forc]), pd.concat([ag,aht_forc])

def grp_df(v,a,g):
    v = v.transpose()
    a = a.transpose()
    vol = round(pd.pivot_table(v, columns = ['Forecast_Grp'], aggfunc = np.sum),0)
    a[a.columns[1:]] = a[a.columns[1:]] * v[v.columns[1:]]
    aht = (round(pd.pivot_table(a, columns = ['Forecast_Grp'], aggfunc = np.sum),0)/vol).fillna(0)
    return vol, aht

def for_and_acc(vol, aht, grp_dict, hols, start_date, 
                 days_to_forecast = 30):
    h = update_hol_lis(hols)
    all_for_vol, all_for_aht = forecast(vol, aht, grp_dict,hols,start_date,days_to_forecast)
    acc_df = pd.DataFrame(index = vol.index[35:], columns = vol.columns).fillna(0)
    for d in acc_df.index:
        if d in hols:
            pass
        elif add_day(d,-1) in h or add_day(d,-2) in h:
            acc_df.loc[d] = round(((((vol.loc[subtract_week(d,h,3)]+
                              2*vol.loc[subtract_week(d,h,2)]+
                              3*vol.loc[subtract_week(d,h,1)])//6)*1.2)-vol.loc[d])/vol.loc[d],2)
            all_for_vol.loc[d] = ((vol.loc[subtract_week(d,h,3)]+
                              2*vol.loc[subtract_week(d,h,2)]+
                              3*vol.loc[subtract_week(d,h,1)])//6)*1.2

        else:
            acc_df.loc[d] = round((((vol.loc[subtract_week(d,h,3)]+
                              2*vol.loc[subtract_week(d,h,2)]+
                              3*vol.loc[subtract_week(d,h,1)])//6)-vol.loc[d])/vol.loc[d],2)
            all_for_vol.loc[d] = ((vol.loc[subtract_week(d,h,3)]+
                              2*vol.loc[subtract_week(d,h,2)]+
                              3*vol.loc[subtract_week(d,h,1)])//6)
    acc_df.index = pd.to_datetime(acc_df.index)
    all_for_vol.index = pd.to_datetime(all_for_vol.index)
    return acc_df, all_for_vol
 
grps = cc.grp_dict()
depts = cc.dept_dict()
holidays = cc.holiday_lis()

vol = clean_daily_vol_aht(file,grps,'Volume')
aht = clean_daily_vol_aht(file,grps,'AHT')

#Take out Intake Surge from the AHTs
aht_adjust = pd.read_csv('dept_10_aht.csv',sep = ',', encoding = 'utf-8')
aht_adjust.row_date = pd.to_datetime(aht_adjust.row_date)
surge = [1290045237,639039222,639045873,1290043335,651053436,1290046227,651055179,1290043554]
aht_adjust = aht_adjust.loc[~aht_adjust.id.isin(surge)]
aht_piv = pd.pivot_table(aht_adjust,
                         values = ['aht','calls_answered'],
                         index = ['row_date'],
                         columns = 'split',
                         aggfunc = {'aht':np.mean,'calls_answered':np.sum},
                         fill_value = 0,
                         margins = False)
aht.loc[aht.index.isin(aht_piv.index), (902,904,909)] = aht_piv.aht*60

v,a = forecast(vol,aht,grps,holidays,start_date = datetime.datetime(2018,1,1),days_to_forecast=30)
vol2, aht2 = grp_df(v,a,grps)

vol_acc, forc = for_and_acc(vol2,aht2,grps,holidays,start_date = datetime.datetime(2018,1,1),days_to_forecast=30)
grouped_accuracy_monthly = vol_acc.groupby(by=depts,axis=1).mean().replace([np.inf, -np.inf], 0).resample('M').mean()

vol2.iloc[-30:]

Forecast_Grp,dept_10a,dept_11a,dept_1a,dept_1b,dept_1c,dept_2a,dept_2b,dept_2c,dept_2d,dept_3a,...,dept_4b,dept_4c,dept_4d,dept_4e,dept_5a,dept_6a,dept_7a,dept_7b,dept_8a,dept_9a
2020-11-18,185.0,1.0,873.0,169.0,321.0,189.0,83.0,175.0,34.0,482.0,...,53.0,335.0,208.0,38.0,107.0,196.0,33.0,128.0,515.0,240.0
2020-11-19,180.0,0.0,789.0,147.0,309.0,215.0,112.0,181.0,23.0,433.0,...,69.0,291.0,177.0,37.0,108.0,169.0,41.0,147.0,459.0,245.0
2020-11-20,214.0,1.0,844.0,147.0,367.0,191.0,106.0,210.0,22.0,447.0,...,77.0,315.0,214.0,39.0,102.0,172.0,36.0,142.0,422.0,248.0
2020-11-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-11-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-11-23,257.0,3.0,1194.0,175.0,535.0,283.0,135.0,266.0,31.0,612.0,...,74.0,382.0,269.0,47.0,127.0,253.0,44.0,201.0,696.0,316.0
2020-11-24,234.0,2.0,1009.0,193.0,392.0,227.0,125.0,231.0,30.0,595.0,...,75.0,353.0,245.0,40.0,130.0,216.0,53.0,151.0,597.0,293.0
2020-11-25,184.0,1.0,865.0,169.0,317.0,185.0,79.0,172.0,32.0,477.0,...,50.0,334.0,209.0,36.0,104.0,191.0,32.0,128.0,511.0,235.0
2020-11-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-11-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
